In [6]:
"""
    Notebook used to develop regressor that uses the entire xgboost_meta_data to predict the AUC

    Everything was moved to fullDataRegression.py and turned into functions
"""
# import pandas as pd
# import numpy as np
# from IPython.core.display_functions import display
# from python.project_utils import get_dataset_to_task
#
# avg_perf = pd.read_csv('./data/average_performance.csv')
# meta_train = pd.read_csv('./data/features.csv')

In [12]:
# meta_train['MaxNominalAttDistinctValues'].hist(bins=100)
# print(meta_train.shape, avg_perf.shape)

(94, 14) (687101, 14)


In [ ]:
# joint_and_averaged = avg_perf.merge(meta_train, how='left', on='data_id')
# joint_and_averaged
# joint_and_averaged.isna().sum()
# joint_and_averaged['MaxNominalAttDistinctValues'] = joint_and_averaged['MaxNominalAttDistinctValues'].fillna(5)
# joint_and_averaged.drop(['index', 'name', 'version', 'status'], axis=1, inplace=True)
# joint_and_averaged['task_id'] = joint_and_averaged['data_id'].map(get_dataset_to_task())

In [27]:
# display(joint_and_averaged)

,data_id,num_round,eta,gamma,lambda,alpha,subsample,max_depth,min_child_weight,colsample_bytree,...,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures,task_id
0,3,1,0.000097,0.002120,1.074280,156.026000,0.202192,2,16.66880,0.442926,...,3.0,1527,2,37,3196,0,0,0,37,3
1,3,1,0.000200,0.014477,0.001188,0.971001,0.186941,7,1.54714,0.644196,...,3.0,1527,2,37,3196,0,0,0,37,3
2,3,1,0.001281,0.000142,0.000156,0.012087,0.367161,12,1.02729,0.223957,...,3.0,1527,2,37,3196,0,0,0,37,3
3,3,1,0.001911,0.021976,0.009044,0.002017,0.876026,14,1.95115,0.355134,...,3.0,1527,2,37,3196,0,0,0,37,3
4,3,1,0.002022,0.000058,0.258957,1.867090,0.354213,4,2.44719,0.239795,...,3.0,1527,2,37,3196,0,0,0,37,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687096,41169,834,0.000271,1.687760,4.597980,9.997210,0.113565,13,1.59306,0.023307,...,100.0,111,100,28,65196,0,0,27,1,168329
687097,41169,834,0.909963,1.115170,18.010900,21499.300000,0.709129,9,31.55750,0.197853,...,100.0,111,100,28,65196,0,0,27,1,168329
687098,41169,863,0.998202,0.000071,260.983000,355.355000,0.327998,8,1.30746,0.679743,...,100.0,111,100,28,65196,0,0,27,1,168329
687099,41169,893,0.359722,0.103639,0.007329,639500.000000,0.218695,2,4.49711,0.826498,...,100.0,111,100,28,65196,0,0,27,1,168329


In [28]:
# joint_and_averaged.to_csv('./data/joint_and_averaged_clean.csv')

In [1]:
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from sklearn.model_selection import RepeatedKFold
import pickle
from datetime import datetime
from skopt.space import Real, Integer
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
import pandas
import pandas as pd
import numpy as np
from IPython.core.display_functions import display
from python.project_utils import get_dataset_to_task, training_meta_features, hyperparameters_data, test_ids
from datetime import datetime
from skopt.space import Real, Integer

full_data = pandas.read_csv('./data/joint_and_averaged_clean.csv')
"""Data will be structured as
X_train: dataset_metafeatures, XGBoost hyperparameters
y_train: avg_auc
"""
sampled_data = full_data.sample(frac=0.15)
display(sampled_data)

def bayes_callback(res):
    print("Next iteration, Time:",datetime.now().strftime("%H:%M:%S"))

,Unnamed: 0,data_id,num_round,eta,gamma,lambda,alpha,subsample,max_depth,min_child_weight,...,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures,task_id
502600,502600,40536,146,0.263461,0.182106,1.347390e+00,0.006482,0.780254,12,8.09592,...,5.0,1380,2,121,8378,7330,18372,59,62,146607
164602,164602,151,311,0.126755,0.000008,7.160270e+01,0.001683,0.904014,11,2.01811,...,7.0,19237,2,9,45312,0,0,7,2,219
128115,128115,46,61,0.213556,1.120790,1.081520e-03,0.003688,0.518079,4,21.59270,...,6.0,767,3,61,3190,0,0,0,61,45
138975,138975,50,390,0.892973,1.968120,1.160000e-06,0.534515,0.716349,1,20.28740,...,3.0,332,2,10,958,0,0,0,10,49
382846,382846,1487,812,0.833905,0.394070,1.878460e+00,12.635700,0.939675,2,62.06850,...,2.0,160,2,73,2534,0,0,72,1,9978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298147,298147,1068,1711,0.170173,0.043988,9.021430e-03,0.000004,0.304678,6,28.07650,...,2.0,77,2,22,1109,0,0,21,1,3918
429918,429918,4134,102,0.006323,0.000731,9.810570e+01,17.321200,0.599755,15,9.16689,...,2.0,1717,2,1777,3751,0,0,1776,1,9910
158052,158052,60,828,0.004691,0.014469,1.771000e+01,120.198000,0.793861,9,4.06040,...,3.0,1653,3,41,5000,0,0,40,1,58
283444,283444,1067,72,0.009739,0.001445,1.760650e+02,0.596482,0.496458,9,50.31140,...,2.0,326,2,22,2109,0,0,21,1,3917


In [16]:
"""
NOTE - Order of features - first dataset features, then hyperparameters
"""

#Defining search space for XGBoost regressor
search_space = {}
search_space['eta'] = Real(0,1,'uniform')
search_space['n_estimators'] = Integer(50,500)
search_space['max_depth'] = Integer(3,15)
search_space['learning_rate'] = Real(10e-4,0.1,'log-uniform')
search_space['colsample_bytree'] = Real(0.3,0.9,'uniform')
search_space['subsample'] = Real(0.3,1,'uniform')

#Selecting relevant features for regressor
training_features = training_meta_features + hyperparameters_data
X = sampled_data[training_features].to_numpy()
y = sampled_data['avg_auc'].to_numpy()

#We split the data 2:1 to get 10% of the entire dataset in each training
cv_outer = RepeatedKFold(n_splits=3, n_repeats=1, random_state=1)
#Lists for nested sampling results
outer_score = []
regressors = []
search_results = []

for counter, (train_index,test_index) in enumerate(cv_outer.split(X,y)):
    print("Starting outer evaluation ",counter+1,"for Bayesian optimisation \n Time:", datetime.now().strftime("%H:%M:%S"))

    #selecting relevant data for fold
    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]

    """
    Inner loop uses 5-fold cross validation to evaluate regressor configurations
    Metric doesn't need to be specified - inherited by XGBoost (standard R2 score)
    """
    cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)
    search = BayesSearchCV(estimator= XGBRegressor(), search_spaces=search_space, n_jobs=-1,\
                           cv=cv, n_iter=20) # n_iter=150
    # perform the search and save results
    curr_result = search.fit(X_train, y_train, callback = bayes_callback)

    search_results.append(curr_result)
    best_candidate = curr_result.best_estimator_
    regressors.append(best_candidate)
    # report the best result
    print("Best score in this iteration:",search.best_score_)
    print("Best regression parameters in this iteration:",search.best_params_)

    y_pred = best_candidate.predict(X_test)
    score = r2_score(y_test, y_pred)
    outer_score.append(score)

    #Save results
    with open('fulldata_xgb_search_results.pkl', 'wb') as f:
        pickle.dump(search_results, f)
    with open('full_data_xgboost_regressors.pkl', 'wb') as f:
        pickle.dump(regressors, f)
    with open('full_data_xgboost_scores.pkl', 'wb') as f:
        pickle.dump(outer_score, f)

Starting outer evaluation  1 for Bayesian optimisation 
 Time: 14:08:13
Next iteration, Time: 14:09:13
Next iteration, Time: 14:10:08
Next iteration, Time: 14:10:38
Next iteration, Time: 14:10:44
Next iteration, Time: 14:11:17
Next iteration, Time: 14:11:45
Next iteration, Time: 14:12:08
Next iteration, Time: 14:14:49
Next iteration, Time: 14:15:42
Next iteration, Time: 14:20:05
Next iteration, Time: 14:25:00
Next iteration, Time: 14:28:20
Next iteration, Time: 14:28:56
Next iteration, Time: 14:32:10
Next iteration, Time: 14:32:24
Next iteration, Time: 14:39:00
Next iteration, Time: 14:39:16
Next iteration, Time: 14:41:29
Next iteration, Time: 14:46:35
Next iteration, Time: 14:48:00
Best score in this iteration: 0.9297696655137203
Best regression parameters in this iteration: OrderedDict([('colsample_bytree', 0.9), ('eta', 0.10088669342732684), ('learning_rate', 0.031810056486861714), ('max_depth', 15), ('n_estimators', 500), ('subsample', 1.0)])
Starting outer evaluation  2 for Bayesi

C:\ProgramData\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Next iteration, Time: 15:30:10
Next iteration, Time: 15:30:22
Next iteration, Time: 15:33:35
Next iteration, Time: 15:34:02
Next iteration, Time: 15:36:05
Next iteration, Time: 15:36:29
Next iteration, Time: 15:39:30
Next iteration, Time: 15:40:36
Next iteration, Time: 15:41:14
Next iteration, Time: 15:47:24
Next iteration, Time: 15:48:57
Next iteration, Time: 15:50:36
Next iteration, Time: 15:57:17
Next iteration, Time: 16:00:58
Next iteration, Time: 16:01:45
Next iteration, Time: 16:07:44
Next iteration, Time: 16:09:43
Next iteration, Time: 16:11:47
Best score in this iteration: 0.9294472488905287
Best regression parameters in this iteration: OrderedDict([('colsample_bytree', 0.9), ('eta', 1.0), ('learning_rate', 0.011757634493083981), ('max_depth', 15), ('n_estimators', 500), ('subsample', 1.0)])


[0.936097452681282, 0.9387437709312048, 0.9353902074991705]
[XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, enable_categorical=False,
             eta=0.10088669342732684, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.031810056486861714,
             max_delta_step=0, max_depth=15, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1.0,
             tree_method='exact', validate_parameters=1, verbosity=None), XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, enable_categorical=False,
             eta=0.0, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.

In [2]:
with open('full_data_xgboost_scores.pkl', 'rb') as f:
    scores = pickle.load(f)
with open('full_data_xgboost_regressors.pkl', 'rb') as f:
    regressors = pickle.load(f)
#with open('fulldata_xgb_search_results.pkl', 'rb') as f:
#    searches = pickle.load(f)



In [3]:
index_max = np.argmax(scores)
best_score = scores[index_max]
best_regressor = regressors[index_max]
#best_search = searches[index_max]
#results_dict = best_search.cv_results_

In [4]:
print(best_regressor)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, enable_categorical=False,
             eta=0.0, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.017764780206487096,
             max_delta_step=0, max_depth=15, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.7538036564590659,
             tree_method='exact', validate_parameters=1, verbosity=None)


In [5]:
test_features = pandas.read_csv('./data/test_features.csv')
display(test_features)


,name,status,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures,data_id,task_id
0,mfeat-karhunen,active,200.0,10.0,200.0,10.0,65.0,2000.0,0.0,0.0,64.0,1.0,16.0,16.0
1,mfeat-zernike,active,200.0,10.0,200.0,10.0,48.0,2000.0,0.0,0.0,47.0,1.0,22.0,22.0
2,credit-g,active,700.0,10.0,300.0,2.0,21.0,1000.0,0.0,0.0,7.0,14.0,31.0,31.0
3,satimage,active,1531.0,6.0,625.0,6.0,37.0,6430.0,0.0,0.0,36.0,1.0,182.0,2074.0
4,eucalyptus,active,214.0,27.0,105.0,5.0,20.0,736.0,95.0,448.0,14.0,6.0,188.0,2079.0
5,monks-problems-2,active,395.0,4.0,206.0,2.0,7.0,601.0,0.0,0.0,0.0,7.0,334.0,3493.0
6,mc1,active,9398.0,2.0,68.0,2.0,39.0,9466.0,0.0,0.0,38.0,1.0,1056.0,3907.0
7,kc2,active,415.0,2.0,107.0,2.0,22.0,522.0,0.0,0.0,21.0,1.0,1063.0,3913.0
8,micro-mass,active,60.0,20.0,11.0,20.0,1301.0,571.0,0.0,0.0,1300.0,1.0,1515.0,9950.0
9,phoneme,active,3818.0,2.0,1586.0,2.0,6.0,5404.0,0.0,0.0,5.0,1.0,1489.0,9952.0


In [9]:
from python.taskRegression import sample_points

"""
For each of the test tasks we will predict the best configuration by sampling random configurations and observing the expected AUC
"""
print("Generating random configurations")
sample_configs = sample_points(n_points=200000)[hyperparameters_data].to_numpy()
display(sample_configs)

Generating random configurations


array([[2.01700000e+03, 6.48260293e-01, 2.87181733e+00, ...,
        5.45600079e+01, 8.26753370e-01, 9.05286405e-01],
       [2.70000000e+01, 5.64719654e-01, 2.20165311e+00, ...,
        4.92511797e+01, 9.88846795e-01, 9.61373654e-01],
       [2.60000000e+01, 6.79016689e-01, 2.35399848e+00, ...,
        2.05648939e+01, 4.18157504e-01, 8.68885101e-01],
       ...,
       [3.60000000e+01, 6.94121098e-01, 2.36656301e-01, ...,
        5.21018298e+01, 1.97559061e-01, 4.22806723e-01],
       [2.40000000e+01, 4.48900958e-01, 1.55533893e+00, ...,
        2.66084441e+01, 7.96993159e-01, 1.52354894e-01],
       [1.92600000e+03, 8.36793962e-02, 3.28433033e+00, ...,
        2.66272690e+01, 7.83051902e-01, 2.73339816e-01]])

In [10]:
from python.baseline import XGBoostTest
from python.project_utils import make_valid_config, meta_feature_names

predicted_config = {}
print("Selecting the best configuration for each task:")
for test_id in test_ids:
    print("Task:",test_id, "time", datetime.now().strftime("%H:%M:%S"))
    meta_featues = test_features[test_features['task_id']==test_id].head(1)[training_meta_features].to_numpy()
    helper_materix = np.tile(meta_featues,(sample_configs.shape[0], 1))
    input = np.concatenate((helper_materix,sample_configs),axis=1)
    predicted_auc = best_regressor.predict(input)
    ind = np.argmax(predicted_auc)
    predicted_config[test_id] = sample_configs[ind,:]

baseline = pd.read_csv('./data/baseline_performance.csv')
comparison = pd.DataFrame()
for key,config in predicted_config.items():
    """For each of the Test tasks (18 in total) we evaluate the performance of its predicted best config for XGBoost"""
    current_task = key
    print("Evaluating task with id: ", current_task)
    seed = 123
    """We save the performance in a dataframe containing:
    1.task_id
    2.baseline auc and time
    3.new auc and time - for the predicted best configuration
    """
    new_row = {}
    new_row['task_id'] = current_task
    base_performance = baseline[baseline['task_id'] == current_task].head(1)
    new_row['base_auc'] = base_performance['auc'].values[0]
    new_row['base_time'] = base_performance['timetrain'].values[0]

    """Corrections to continuous predictions that might not be possible"""
    new_config = dict(zip(hyperparameters_data, config))
    new_config = make_valid_config(new_config)

    """Evaluating new config performance"""
    objective = XGBoostTest(current_task, meta_feature_names=meta_feature_names, seed=seed)
    auc, traintime = objective.evaluate(new_config)

    """Adding info about new performance """
    new_row['new_auc'] = auc
    print("New auc", auc)
    new_row['new_time'] = traintime
    new_row['delta_auc'] = new_row['new_auc'] - new_row['base_auc']
    new_row['better_auc'] = new_row['new_auc'] > new_row['base_auc']
    new_row['better_time'] = new_row['new_time'] < new_row['base_time']
    new_row = pd.DataFrame([new_row])
    comparison = pd.concat([comparison, new_row], ignore_index=True)
display(comparison)
comparison.to_csv('./data/fulldata_nested_xgboost_results.csv')


Selecting the best configuration for each task:
Task: 16 time 18:47:26
Task: 22 time 18:47:30
Task: 31 time 18:47:34
Task: 2074 time 18:47:37
Task: 2079 time 18:47:40
Task: 3493 time 18:47:44
Task: 3907 time 18:47:47
Task: 3913 time 18:47:51
Task: 9950 time 18:47:54
Task: 9952 time 18:47:57
Task: 9971 time 18:48:01
Task: 10106 time 18:48:04
Task: 14954 time 18:48:07
Task: 14970 time 18:48:12
Task: 146212 time 18:48:15
Task: 167119 time 18:48:19
Task: 167125 time 18:48:22
Task: 168336 time 18:48:25
Evaluating task with id:  16


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:48:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
New auc 0.9945
Evaluating task with id:  22


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:48:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
New auc 0.9688333333333332
Evaluating task with id:  31


...

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:11:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
New auc 0.9657338883749614
Evaluating task with id:  168336


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:13:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
New auc 0.7526029479664404


,task_id,base_auc,base_time,new_auc,new_time,delta_auc,better_auc,better_time
0,16,0.997222,31.225133,0.994500,14.023385,-0.002722,False,True
1,22,0.973361,25.774230,0.968833,24.805384,-0.004528,False,True
2,31,0.841905,1.274992,0.810000,2.276000,-0.031905,False,False
3,2074,0.989416,18.240999,0.989734,40.257960,0.000319,True,False
4,2079,0.915436,3.369999,0.887997,1.475038,-0.027439,False,True
5,3493,0.976190,0.539915,0.500000,0.354961,-0.476190,False,True
6,3907,0.985866,2.770957,0.901596,0.324998,-0.084271,False,True
7,3913,0.906926,0.559001,0.898268,1.623962,-0.008658,False,False
8,9950,0.996221,32.786961,0.991830,2.706999,-0.004390,False,True
9,9952,0.956024,2.991995,0.953802,3.121042,-0.002223,False,False
